<a href="https://colab.research.google.com/github/Aloncohen41/LOLProject/blob/main/LoL_Champion_Kill_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import requests
import os
from google.colab import userdata
import json
import time # Import the time module for rate limiting

# Replace with your actual API key from Colab Secrets
RIOT_API_KEY = userdata.get('PERSONAL_RIOT_KEY') # Using the new key name

# Replace with the appropriate region for the matches (e.g., 'na1', 'euw1', 'kr')
# This should be the region where the matches were played.
# YOU MUST REPLACE 'YOUR_REGION' BELOW WITH A VALID REGION CODE!
region = 'europe' # <--- Change this to a valid region

# Path to the file containing match IDs
match_ids_file = '/content/game_ids.txt'

# Directory to save the match data
output_dir = '/content/match_data'
os.makedirs(output_dir, exist_ok=True)

# Set up the headers with your API key
headers = {
    'X-Riot-Token': RIOT_API_KEY
}

# Rate limiting: Riot API has rate limits. Be mindful of them.
# You might need to adjust this based on your API key's limits.
# A simple approach is to wait between requests.
request_delay = 1 # seconds between requests

match_ids = []
try:
    with open(match_ids_file, 'r') as f:
        match_ids = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print(f"Error: The file '{match_ids_file}' was not found.")
    match_ids = []

if not match_ids:
    print("No match IDs found in the file. Please ensure the file exists and contains match IDs.")
else:
    print(f"Found {len(match_ids)} match IDs in the file. Starting data extraction...")
    for i, match_id in enumerate(match_ids):
        api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}'
        output_file = os.path.join(output_dir, f'{match_id}.json')

        # Check if the file already exists to avoid re-fetching
        if os.path.exists(output_file):
            print(f"Skipping match ID {match_id}: data already exists.")
            continue

        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status() # Raise an exception for bad status codes

            match_data = response.json()

            # Save the match data to a JSON file
            with open(output_file, 'w') as f:
                json.dump(match_data, f, indent=4)

            print(f"Successfully extracted and saved data for match ID {match_id} ({i+1}/{len(match_ids)})")

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for match ID {match_id}: {e}")
            if 'response' in locals():
                if response.status_code == 401:
                    print("Unauthorized: Check if your API key is valid and correctly set.")
                elif response.status_code == 403:
                    print("Forbidden: Your API key may not have the necessary permissions or has expired.")
                elif response.status_code == 404:
                    print("Not Found: Match ID not found or invalid.")
                elif response.status_code == 429:
                    print("Rate Limit Exceeded: You have made too many requests in a short period. Waiting...")
                    # Implement a backoff strategy for rate limiting
                    time.sleep(60) # Wait for 60 seconds and retry or adjust delay
                    continue # Skip to the next match for now, or implement retry logic
                else:
                    print(f"API returned status code: {response.status_code}")
            else:
                print("Request failed before receiving a response.")

        # Wait for the specified delay to respect rate limits
        time.sleep(request_delay)

    print("Match data extraction process finished.")

Found 51490 match IDs in the file. Starting data extraction...
Error fetching data for match ID 3326086514: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/3326086514
Not Found: Match ID not found or invalid.
Error fetching data for match ID 3229566029: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/3229566029
Not Found: Match ID not found or invalid.
Error fetching data for match ID 3327363504: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/3327363504
Not Found: Match ID not found or invalid.
Error fetching data for match ID 3326856598: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/3326856598
Not Found: Match ID not found or invalid.
Error fetching data for match ID 3330080762: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/3330080762
Not Found: Match ID not found or i

KeyboardInterrupt: 

**Note:** You will need a Riot API key with access to the Account-v1 endpoint to use this code.

Here's how to find your PUUID using the Riot Games API Documentation website:

1.  Go to the [Riot Games API Documentation website](https://developer.riotgames.com/).
2.  Log in with your Riot Games account.
3.  Navigate to the **API Reference** section.
4.  Find and click on the **Account-v1** API.
5.  Look for the endpoint that allows you to get an account by Riot ID. It should be something like `/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}`.
6.  Click on this endpoint to expand its details.
7.  In the "Parameters" section, you will see fields for `gameName` (your summoner name) and `tagLine` (your tag line).
8.  Enter your summoner name in the `gameName` field and your tag line in the `tagLine` field.
9.  Make sure you have selected the correct **Region** (usually 'americas', 'europe', or 'asia' depending on where your account was created).
10. Click the **Execute** button.
11. The response will appear below. Look for the `puuid` field in the JSON response. The value associated with this field is your PUUID.

Remember to replace `YOUR_SUMMONER_NAME` and `YOUR_TAG_LINE` with your actual in-game name and tag line.